# 1. 월별로 분리되어 있는 데이터프레임을 하나로 합치는 과정

In [ ]:
raw_df = pd.DataFrame() # 1월부터 6월까지의 데이터 프레임을 합치기 위해 빈 데이터프레임 생성
for month in range(1, 7): # 1월부터 6월까지 csv파일을 for문으로 읽어서 데이터 프레임을 concat()을 이용하여 합치는 과정
    temp_df = pd.read_csv(f'rawfiles/CARD_SUBWAY_MONTH_20190{month}.csv')
    raw_df = pd.concat([raw_df, temp_df], axis=0) # axis=0은 행 기준(axis=1은 열 기준). 행을 따라(0행부터 아래 행으로 이동하면서) 데이터 프레임을 꽂는 느낌.

raw_df = raw_df.reset_index(drop=True)  # concat으로 데이터 프레임을 합치면, 데이터프레임 각각의 인덱스가 유지됨. 인덱스가 중복되게 되므로 reset_index()로 인덱스를 0번부터 차곡차곡 쌓이도록 재설정.

# 주어진 문제 해결을 위해 사용일자를 datetime으로 형변환 후 요일, 연월, 월일 컬럼 추가

In [ ]:
raw_df['사용일자'] = pd.to_datetime(raw_df['사용일자'], format='%Y%m%d') # 사용일자의 자료형을 int에서 datatime으로 형변환

In [ ]:
raw_df['요일'] = raw_df['사용일자'].dt.dayofweek   # 컬럼으로 "요일"을 추가. dt.dayofweek는 0(월) ~ 6(일) 로 만들어줌.

In [ ]:
day_of_week = {  # 요일이 숫자이므로 이를 문자로 바꾸기 위해 딕셔너리를 이용.
    0 : '월',
    1 : '화',
    2 : '수',
    3 : '목',
    4 : '금',
    5 : '토',
    6 : '일'
}
raw_df['요일'] = raw_df['요일'].map(day_of_week)  # "요일" 컬럼의 data를 map()을 이용하여 숫자로 된 요일이 각각 해당하는 문자에 대응하도록 변환.

In [ ]:
raw_df['연월'] = raw_df['사용일자'].dt.strftime('%Y-%m') # 컬럼으로 "연월"을 추가. dt.strftime()은 datetime이 아니라 문자열을 반환.
raw_df['월일'] = raw_df['사용일자'].dt.strftime('%m-%d') # 컬럼으로 "월일"을 추가. dt.strftime()은 datetime이 아니라 문자열을 반환.
raw_df.head()

# 주어진 문제 해결을 위해 승차총승객수와 하차총승객수를 합하여 승하차총승객수 컬럼 추가

In [ ]:
raw_df['승하차총승객수'] = raw_df['승차총승객수'] + raw_df['하차총승객수'] # 컬럼으로 "승하차총승객수"를 추가.
raw_df = raw_df[['사용일자', '요일', '노선명', '역명', '승차총승객수', '하차총승객수', '승하차총승객수', '연월', '월일']] # data분석에 필요한 컬럼을 모두 만들었으므로 컬럼 재정렬.
raw_df.head()

# NaN값 있는지 확인. 있다면 처리했어야 했지만 없어서 pass

In [ ]:
# NaN값 있는지 확인했으나 없었음 -> pass

# '사용일자','요일', '노선명', '역명', '연월', '월일'을 기준으로 중복 데이터가 있는지 확인.

 이 컬럼을 기준으로 잡은 이유: 같은 날에 지하철역과 그 역의 노선도 같다면,,,  같은 날, 같은 역, 같은 호선에서 집계된 결과이기 때문.

In [ ]:
# 중복데이터가 있는지 확인하는 코드
raw_df.duplicated(['사용일자','요일', '노선명', '역명', '연월', '월일'], keep=False).sum() # duplicated()로 중복 데이터 확인. 매개변수로 기준이 될 컬럼(여러 개면 리스트로 넣음)을 넣어주면 됨.
                                                                                           # 기준이 된 컬럼들에 대한 data가 모두 같을 때만 중복데이터임. 그 중복데이터들의 개수을 구해봄.
                                                                                           # keep=False는 gpt검색해보면 쉽게 알 수 있음. keep=False면 중복된 데이터를 모두 포함시킴.

하나만 남기고 제거하지 않고 합친 이유: 서울 열린데이터 광장에서 검색 결과 집계가 어느정도 될때마다 데이터를 등록한다고 함. 등록 후에는 승객수를 0명부터 다시 데이터를 집계하기 시작.

In [ ]:
raw_df = raw_df.groupby(['사용일자', '요일','노선명','역명','연월','월일']).sum().reset_index()